Step by step demonstration of all functions of the BrainOvservatoryCache Class

First Get the proper path to store the manifest and metadata and the downloade nwb files

In [1]:
from pathlib import Path
data_folder = Path("C:/Users/Samuel/Documents/PythonCodeData/BrainObservatory")

Next create the manifest and the folder

In [2]:
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import pprint
# This class uses a 'manifest' to keep track of downloaded data and metadata.  
# All downloaded files will be stored relative to the directory holding the manifest
# file.  If 'manifest_file' is a relative path (as it is below), it will be 
# saved relative to your working directory.  It can also be an absolute path.
boc = BrainObservatoryCache(manifest_file=data_folder/'boc/manifest.json')

ModuleNotFoundError: No module named 'allensdk'

In [ ]:
import pprint

We can rebuild the manifest in another folder. This does not creat new folders

In [ ]:
boc.build_manifest(data_folder/'boc/manifest2.json')

Now lets get all the info on the different parameters from the observatory

In [ ]:
cre=boc.get_all_cre_lines() #global information about the databse to have it in lists
area=boc.get_all_targeted_structures() #global information about the databse to have it in lists
depth=boc.get_all_imaging_depths()#global information about the databse to have it in lists
gcamp=boc.get_all_reporter_lines()#global information about the databse to have it in lists GCAMP used
sessions=boc.get_all_session_types()#global information about the databse to have it in lists
stimuli=boc.get_all_stimuli()#global information about the databse to have it in lists

Not lets see those lists

In [ ]:
print(cre)
print(area)
print(depth)
print(gcamp)
print(sessions)
print(stimuli)

Not lets get some container with different filters, and change some of the parameters
First just using one filter. By the way some of the previous functions already donlades the experiments containes and experiemnst metadata


In [ ]:
visP=boc.get_experiment_containers(targeted_structures=['VISp'])
pprint.pprint(visP[3])
print(len(visP))

This has not added any new file lets try the saving option. I thinks is yjust saving a smaller filter json file 
It saves the whole container metadata again
Remember I am in the path of the code, not of the data so ll paths have to be absolute here

In [ ]:
visP=boc.get_experiment_containers(file_name='boc/test.json', targeted_structures=['VISp'])
pprint.pprint(visP[3])
print(len(visP))

Now lest play with the simplify option. It gets informatio0n that is relevant form other databases and more 

In [ ]:
visP=boc.get_experiment_containers(targeted_structures=['VISp'], simple=False)
pprint.pprint(visP[3])
print(len(visP))

No lets try to use two filters
It seems you cna only filter by one. Lets try anotehr thing
It allows, I just put a string instead of an integer
It works in differnet ways, but cannot filter by stimuli or stimuli seesion, those are only ofr experimenst

In [ ]:
visPLayer2=boc.get_experiment_containers(targeted_structures=['VISp'], imaging_depths=[175])
pprint.pprint(visPLayer2[0])
print(len(visPLayer2))

Lets try getting all Id from the dictionary and parsing that with another parameter

In [ ]:
visP=boc.get_experiment_containers(targeted_structures=['VISp'])
contIDs=[cont['id']for cont in visP]
contIDs
visP[0]['id']

In [ ]:
visPSpon=boc.get_experiment_containers(ids=contIDs, reporter_lines=['Ai148(TIT2L-GC6f-ICL-tTA2)', 'Ai162(TIT2L-GC6s-ICL-tTA2)'])
pprint.pprint(visPSpon[0])
print(len(visPSpon))

That´s all I will get from the containers. A series of IDs to then restric experimet. But It can Actually be done directly on get experiments as we will se below
Lets get first VISp. 275, PV cells.
It gives 7 differnet animals tha were recorderd
Lets get their containers IDS. One animal could have several conatiners

In [ ]:
visPLy2PC=boc.get_experiment_containers(ids=contIDs, cre_lines=['Pvalb-IRES-Cre'], imaging_depths=[275], reporter_lines=['Ai162(TIT2L-GC6s-ICL-tTA2)'])
pprint.pprint(visPLy2PC[0])
print(len(visPLy2PC))
PV_V1_L2_contIDs=[cont['id']for cont in visPLy2PC]


Let´s try to find all coantainers for a single animal
Now lets filter only those that have a givewn donor name
So this animal has two experiments
Basically the imaged two depths in this animal, but we dont know if they had the same visual stimuli. 
That we can only know parisng experiments

In [ ]:
AllCont=boc.get_experiment_containers()
len(AllCont)
SingleDOnor_donorIDs=[cont['donor_name']for cont in AllCont if cont['donor_name']=='369498']
SingleDOnor_contIDs=[cont['id']for cont in AllCont if cont['donor_name']=='369498']
print(SingleDOnor_contIDs)
print(SingleDOnor_donorIDs)
SingleDOnorCOnt=boc.get_experiment_containers(ids=SingleDOnor_contIDs)
pprint.pprint(SingleDOnorCOnt)

NOw we move to extract the experiments from these two containers and try to get the same experiments without getting the container IDS
As expected 6 experiments, three for each container
We can get those experiments IDs
And we can check th stimuli they recived, it can only be done for single experiments

In [ ]:
exps=boc.get_ophys_experiments(experiment_container_ids=SingleDOnor_contIDs)
pprint.pprint(exps)
exps_ids=[exp['id'] for exp in exps]
pprint.pprint(exps_ids)
exp_stimuli=[boc.get_ophys_experiment_stimuli(i) for i in exps_ids ]
pprint.pprint(exp_stimuli)
exp_session=[exp['session_type'] for exp in exps ]
pprint.pprint(exp_session)

Now get the same directly with experiment arguments. First get the 7 pv videops I got at 275 and then the specific of the same donor. Finally I want to get all the other layers form all those 7 donors and get some list classified by 
    Session
    Layer
    Donor


In [ ]:
my_area=['VISp']
my_layer=[275]
my_cre=['Pvalb-IRES-Cre']
exps=boc.get_ophys_experiments(targeted_structures=my_area, imaging_depths=my_layer, cre_lines=my_cre)
print(len(exps))
exp_donor_ids=[exp['donor_name']for exp in exps]
print(exp_donor_ids)

list_set = set(exp_donor_ids) 
# convert the set to the list 
unique_list = (list(list_set)) 
print(unique_list)

AllCont=boc.get_experiment_containers()
ExpSingleDOnor_donorIDs=[cont['donor_name'] for cont in AllCont if any([cont['donor_name']==donname for donname in unique_list])]
ExpSingleDOnor_ContIDs=[cont['id'] for cont in AllCont if any([cont['donor_name']==donname for donname in unique_list])]
pprint.pprint(ExpSingleDOnor_donorIDs)

Exp_Cont_Organized=[boc.get_experiment_containers(experiment_container_ids=ExpSingleDOnor_ContIDs)]
donor_list=[]
for i in unique_list:
    donor_list.append(donor_conts=[cont['id'] for cont in AllCont if i==cont['donor_name']])



In [ ]:
donor_list={}
for i in unique_list:
    donor_conts=[cont['id'] for cont in AllCont if i==cont['donor_name']]
    donor_list[i]=donor_conts
donor_list